In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('/content/enhanced_retail_data.db')
cursor = conn.cursor()

# Get the list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in the database:")
for table_name in tables:
    table_name = table_name[0] # Extract table name from the tuple
    print(f"\n--- Data in table: {table_name} ---")
    try:
        # Query the first 5 rows of the table
        query = f"SELECT * FROM {table_name} LIMIT 5;"
        cursor.execute(query)
        rows = cursor.fetchall()

        if rows:
            # Get column names
            column_names = [description[0] for description in cursor.description]
            # Create a pandas DataFrame for better display
            df = pd.DataFrame(rows, columns=column_names)
            display(df)
        else:
            print("Table is empty.")
    except Exception as e:
        print(f"Error querying table {table_name}: {e}")


# Close the connection
conn.close()

Tables in the database:

--- Data in table: historical_data ---


,id,product,category,date,price,cost,promotion,seasonality,sales,sales_value,profit,price_elasticity,profit_margin
0,1,t-shirt,men,2023-06-01 00:00:00,1021.0,592.0,20.0,1.1,65.0,66365.0,27885.0,-0.86,0.42
1,1,t-shirt,men,2023-07-01 00:00:00,1027.0,592.0,20.0,1.1,64.0,65728.0,27840.0,-0.86,0.42
2,1,t-shirt,men,2023-08-01 00:00:00,1276.0,592.0,0.0,1.1,43.0,54868.0,29412.0,-0.86,0.54
3,1,t-shirt,men,2023-09-01 00:00:00,1280.0,592.0,0.0,0.9,35.0,44800.0,24080.0,-0.86,0.54
4,1,t-shirt,men,2023-10-01 00:00:00,1273.0,592.0,0.0,0.9,35.0,44555.0,23835.0,-0.86,0.53



--- Data in table: forecast_data ---


,id,product,category,date,sales
0,1,t-shirt,men,2025-09-01 00:00:00,47.0
1,1,t-shirt,men,2025-10-01 00:00:00,47.0
2,1,t-shirt,men,2025-11-01 00:00:00,47.0
3,1,t-shirt,men,2025-12-01 00:00:00,47.0
4,1,t-shirt,men,2026-01-01 00:00:00,47.0



--- Data in table: inventory_data ---


,id,product,category,stock
0,1,t-shirt,men,141.0
1,8,t-shirt,women,64.0



--- Data in table: competitior_price ---


,id,product,category,competitor_price,competitor_promotion,price_change
0,1,t-shirt,men,1009.4,BTGO,0.7
1,8,t-shirt,women,1382.0,0,1.0


In [3]:
!pip install -q langchain langchain-groq python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 6.0 MB/s eta 0:00:00


In [4]:
# ============================================
# 🔹 Enter your Groq API key securely
# ============================================
import os
from getpass import getpass

os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [5]:
from langchain_groq import ChatGroq
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.memory import ConversationBufferMemory
from langchain.tools import StructuredTool
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from pydantic.v1 import BaseModel, Field
from math import sqrt
import os
from dotenv import load_dotenv

# --- Load API key ---
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# --- Tool schema ---
class HistoricalData(BaseModel):
    number: float = Field(..., description="A non-negative number")

def query_historical_data(number: float) -> float:
    """Calculate the square root of a non-negative number."""
    if number < 0:
        raise ValueError("Cannot compute the square root of a negative number.")
    return sqrt(number)

HistoricalDataTool = StructuredTool.from_function(
    func=square_root,
    name="square_root",
    description="Returns answer to any questions related to historical retail data",
    args_schema=HistoricalData,
)

# --- LLM ---
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
#llm=ChatGroq(model="llama-3.3-70b-versatile", temperature=0)

# --- Memory ---
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="input",
    return_messages=True
)

# --- Prompt ---
system_message = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant, but you are terrible at math. "
    "Whenever asked to compute or find a square root, you MUST call the tool `square_root`."
)

human_message = HumanMessagePromptTemplate.from_template("{input}")

prompt = ChatPromptTemplate.from_messages([
    system_message,
    MessagesPlaceholder(variable_name="chat_history"),
    human_message,
    MessagesPlaceholder(variable_name="agent_scratchpad"),  # required
])

# --- Agent ---
agent = create_tool_calling_agent(
    llm=llm,
    tools=[square_root_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=[square_root_tool],
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True,  # 👈 helpful for debugging
)

# --- Run loop ---
if __name__ == "__main__":
    print("Agent ready! Ask me something (type 'exit' to quit):")
    while True:
        query = input(">> ")
        if query.lower() in ["exit", "quit", "q"]:
            print("Goodbye 👋")
            break
        try:
            response = agent_executor.invoke({"input": query})
            print("🤖:", response.get("output"))
            print("🛠 Steps:", response.get("intermediate_steps"))  # 👈 debug
        except Exception as e:
            print("❌ Error:", e)

/tmp/ipython-input-1120465292.py:37: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Agent ready! Ask me something (type 'exit' to quit):
>> give me the square root of 4


> Entering new AgentExecutor chain...

Invoking: `square_root` with `{'number': 4}`


2.0

/usr/local/lib/python3.12/dist-packages/langchain/chains/base.py:487: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  self.memory.save_context(inputs, outputs)


The square root of 4 is 2.0.

> Finished chain.
🤖: The square root of 4 is 2.0.
🛠 Steps: [(ToolAgentAction(tool='square_root', tool_input={'number': 4}, log="\nInvoking: `square_root` with `{'number': 4}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': '0k5ffp2v7', 'function': {'arguments': '{"number":4}', 'name': 'square_root'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7b3cfae3af', 'service_tier': 'on_demand'}, id='run--0fcc5fa3-0924-4088-8175-d28f908ac313', tool_calls=[{'name': 'square_root', 'args': {'number': 4}, 'id': '0k5ffp2v7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 269, 'output_tokens': 14, 'total_tokens': 283}, tool_call_chunks=[{'name': 'square_root', 'args': '{"number":4}', 'id': '0k5ffp2v7', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='0k5ffp2v7'), 2.0)]
>> exit
Goodbye 👋


# Task
Create a text-to-SQL agent using Langchain and Groq to query the "enhanced_retail_data.db" database.

## Define a tool for database querying

### Subtask:
Create a Langchain tool that can execute SQL queries against the `enhanced_retail_data.db` database.


**Reasoning**:
Define a function to execute SQL queries and create a Langchain tool from it to query the database.



In [19]:
from langchain.tools import StructuredTool
import sqlite3
from pydantic.v1 import BaseModel, Field

class SqlQuery(BaseModel):
    query: str = Field(..., description="A SQL query to execute against the database.")

def execute_sql_query(query: str) -> str:
    """Executes a SQL query against the enhanced_retail_data.db database and returns the results."""
    try:
        conn = sqlite3.connect('/content/enhanced_retail_data.db')
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        column_names = [description[0] for description in cursor.description]
        conn.close()

        if not rows:
            return "Query returned no results."

        # Format the results for readability
        results = [dict(zip(column_names, row)) for row in rows]
        return str(results)
    except Exception as e:
        return f"Error executing SQL query: {e}"

SqlQueryTool = StructuredTool.from_function(
    func=execute_sql_query,
    name="sql_query",
    description="Executes a SQL query against the enhanced_retail_data.db database. Use this tool to answer questions about the retail data.",
    args_schema=SqlQuery,
)

## Initialize the llm

### Subtask:
Set up the Groq language model to be used by the agent.


**Reasoning**:
Import the ChatGroq class and instantiate it to set up the language model for the agent.



In [20]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)

## Create the agent

### Subtask:
Define the agent using the LLM, the database querying tool, and a prompt that instructs the agent to convert natural language questions into SQL queries.


**Reasoning**:
Define the prompt templates and the agent using the initialized LLM and the previously created SQL query tool.



In [21]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

# --- Memory ---
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="input",
    return_messages=True
)

# --- Prompt ---
system_message = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant that can answer questions about retail data. "
    "Your task is to translate user questions into SQL queries and execute them using the `sql_query` tool. "
    "Strictly adhere to the provided database schema when constructing your queries. "
    "Only use the `sql_query` tool. Do not use any other tools."
    "When asked about units sold, use the 'sales' column from the 'historical_data' table."
    "When asked about inventory or stock, use the 'stock' column from the 'inventory_data' table."
    "When asked about competitor's price, use the 'competitor_price' column from the 'competitor_price' table."
    "When asked about price elasticity, use the 'price_elasticity' column from the 'historical_data' table."
    "When referring to products, use the exact 'product' name as it appears in the database, for example 't-shirt' instead of 't shirt'."
    "\n\nDatabase Schema:\n"
    "Table: historical_data\n"
    "Columns: id, product, category, date, price, cost, promotion, seasonality, sales, sales_value, profit, price_elasticity, profit_margin\n\n"
    "Table: forecast_data\n"
    "Columns: id, product, category, date, sales\n\n"
    "Table: inventory_data\n"
    "Columns: id, product, category, stock\n\n"
    "Table: competitior_price\n"
    "Columns: id, product, category, competitor_price, competitor_promotion, price_change\n"
)

human_message = HumanMessagePromptTemplate.from_template("{input}")

prompt = ChatPromptTemplate.from_messages([
    system_message,
    MessagesPlaceholder(variable_name="chat_history"),
    human_message,
    MessagesPlaceholder(variable_name="agent_scratchpad"),  # required
])

# --- Agent ---
agent = create_tool_calling_agent(
    llm=llm,
    tools=[SqlQueryTool],
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=[SqlQueryTool],
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
)

## Set up memory

### Subtask:
Configure conversation memory for the agent.


**Reasoning**:
Instantiate `ConversationBufferMemory` with the specified parameters.



In [22]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="input",
    return_messages=True
)

## Run the agent

### Subtask:
Implement a loop to interact with the agent, allowing you to ask questions about the database in natural language.


**Reasoning**:
Implement the interaction loop for the agent.



In [ ]:
if __name__ == "__main__":
    print("Ask me something about the retail data (type 'exit' to quit):")
    while True:
        query = input(">> ")
        if query.lower() in ["exit", "quit", "q"]:
            print("Goodbye 👋")
            break
        try:
            response = agent_executor.invoke({"input": query})
            print("🤖:", response.get("output"))
            print("🛠 Steps:", response.get("intermediate_steps"))
        except Exception as e:
            print("❌ Error:", e)

Ask me something about the retail data (type 'exit' to quit):
>> give me the number of men's t shirts are persent in inventory


> Entering new AgentExecutor chain...

Invoking: `sql_query` with `{'query': "SELECT stock FROM inventory_data WHERE product = 'men's t shirt'"}`


Error executing SQL query: near "s": syntax error
Invoking: `sql_query` with `{'query': "SELECT stock FROM inventory_data WHERE product = 'men''s t shirt'"}`


Query returned no results.
Invoking: `sql_query` with `{'query': "SELECT stock FROM inventory_data WHERE product = 'men''s t shirt'"}`


Query returned no results.
Invoking: `sql_query` with `{'query': "SELECT SUM(stock) FROM inventory_data WHERE product = 'men''s t shirt'"}`


[{'SUM(stock)': None}]
Invoking: `sql_query` with `{'query': "SELECT SUM(stock) FROM inventory_data WHERE product = 'men''s t shirt'"}`


[{'SUM(stock)': None}]
Invoking: `sql_query` with `{'query': "SELECT SUM(stock) FROM inventory_data WHERE product = 'men''s t shirt'"}`


[{'SUM(s

/usr/local/lib/python3.12/dist-packages/langchain/chains/base.py:487: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  self.memory.save_context(inputs, outputs)



Invoking: `sql_query` with `{'query': "SELECT SUM(stock) FROM inventory_data WHERE product = 'men''s t shirt'"}`


[{'SUM(stock)': None}]

> Finished chain.
🤖: Agent stopped due to max iterations.
🛠 Steps: [(ToolAgentAction(tool='sql_query', tool_input={'query': "SELECT stock FROM inventory_data WHERE product = 'men's t shirt'"}, log='\nInvoking: `sql_query` with `{\'query\': "SELECT stock FROM inventory_data WHERE product = \'men\'s t shirt\'"}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': '6zhcebtff', 'function': {'arguments': '{"query":"SELECT stock FROM inventory_data WHERE product = \'men\'s t shirt\'"}', 'name': 'sql_query'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7b3cfae3af', 'service_tier': 'on_demand'}, id='run--658cb385-4c64-489a-954a-1c0a9d516998', tool_calls=[{'name': 'sql_query', 'args': {'query': "SELECT stock FROM inve

/usr/local/lib/python3.12/dist-packages/langchain/chains/base.py:487: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  self.memory.save_context(inputs, outputs)


You have 1 product in the inventory, which is 't-shirt'.

> Finished chain.
🤖: You have 1 product in the inventory, which is 't-shirt'.
🛠 Steps: [(ToolAgentAction(tool='sql_query', tool_input={'query': 'SELECT COUNT(DISTINCT product) FROM inventory_data'}, log="\nInvoking: `sql_query` with `{'query': 'SELECT COUNT(DISTINCT product) FROM inventory_data'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'sd5rgqyv1', 'function': {'arguments': '{"query":"SELECT COUNT(DISTINCT product) FROM inventory_data"}', 'name': 'sql_query'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ab04adca7d', 'service_tier': 'on_demand'}, id='run--3d75a900-3e65-42ee-9183-9389da86bba9', tool_calls=[{'name': 'sql_query', 'args': {'query': 'SELECT COUNT(DISTINCT product) FROM inventory_data'}, 'id': 'sd5rgqyv1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 575, '

/usr/local/lib/python3.12/dist-packages/langchain/chains/base.py:487: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  self.memory.save_context(inputs, outputs)


You have the following products in the inventory:

- Men's t-shirt: 141 units
- Women's t-shirt: 64 units

> Finished chain.
🤖: You have the following products in the inventory:

- Men's t-shirt: 141 units
- Women's t-shirt: 64 units
🛠 Steps: [(ToolAgentAction(tool='sql_query', tool_input={'query': "SELECT product, category, stock FROM inventory_data WHERE category != 'men's t-shirt'"}, log='\nInvoking: `sql_query` with `{\'query\': "SELECT product, category, stock FROM inventory_data WHERE category != \'men\'s t-shirt\'"}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': '837qxzbpa', 'function': {'arguments': '{"query":"SELECT product, category, stock FROM inventory_data WHERE category != \'men\'s t-shirt\'"}', 'name': 'sql_query'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ab04adca7d', 'service_tier': 'on_demand'}, id='run--c2b0f00e-ed77-4

/usr/local/lib/python3.12/dist-packages/langchain/chains/base.py:487: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  self.memory.save_context(inputs, outputs)



Invoking: `sql_query` with `{'query': "SELECT product, category, competitor_price FROM competitor_prices_table WHERE product = 't-shirt'"}`


Error executing SQL query: no such table: competitor_prices_table

> Finished chain.
🤖: Agent stopped due to max iterations.
🛠 Steps: [(ToolAgentAction(tool='sql_query', tool_input={'query': "SELECT product, category, competitor_price FROM competitor_price WHERE product = 't-shirt'"}, log='\nInvoking: `sql_query` with `{\'query\': "SELECT product, category, competitor_price FROM competitor_price WHERE product = \'t-shirt\'"}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': '5wypj812e', 'function': {'arguments': '{"query":"SELECT product, category, competitor_price FROM competitor_price WHERE product = \'t-shirt\'"}', 'name': 'sql_query'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_33e8adf159', 'servic